In [ ]:
# Downloading required libraries
!pip install -q transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31.3 MB/s eta 0:00:00


In [ ]:
# Deleting the cache content of the transformer namespace
!rm -rf $HOME/.cache/huggingface/
!mkdir $HOME/.cache/huggingface/

In [ ]:
# Downloading the pre-trained t5 model
!transformers-cli download t5-small

2023-04-16 06:56:04.446686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Load the required libraries
# from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import torch

In [ ]:
# Load the pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Load the SQuAD v2 dataset
dataset = load_dataset('squad_v2')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Extract the passages 
passages = dataset['train']['context']

In [ ]:
# Extract the answers
answers=[]
answer_array = dataset['train']['answers']
for i in answer_array:
  if(len(i['text']) != 0):
    answers.append(i['text'][0])
  else:
    answers.append("")

In [ ]:
# Prepare the input and output for the T5 model
inputs = []
outputs = []
for i in range(len(passages)):
    input_text = f'generate questions: "{passages[i]}" answer: "{answers[i]}"'
    output_text = f'{dataset["train"][i]["question"]}'
    inputs.append(input_text)
    outputs.append(output_text)

In [ ]:
# Tokenize the input and output using the T5 tokenizer
input_ids = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')['input_ids']
output_ids = tokenizer(outputs, padding=True, truncation=True, return_tensors='pt')['input_ids']

In [ ]:
# Fine-tune the T5 model on the SQuAD v2 dataset
# Checking if GPU is available and switching model to GPU Runtime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
model.train()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
total_length_of_input = len(input_ids)
for i in range(len(input_ids)):
    input_ids_batch = input_ids[i].unsqueeze(0).to(device)
    output_ids_batch = output_ids[i].unsqueeze(0).to(device)
    loss = model(input_ids_batch, labels=output_ids_batch).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Running " + str(i) + " of " + str(total_length_of_input))

Streaming output truncated to the last 5000 lines.
Running 125319 of 130319
Running 125320 of 130319
Running 125321 of 130319
Running 125322 of 130319
Running 125323 of 130319
Running 125324 of 130319
Running 125325 of 130319
Running 125326 of 130319
Running 125327 of 130319
Running 125328 of 130319
Running 125329 of 130319
Running 125330 of 130319
Running 125331 of 130319
Running 125332 of 130319
Running 125333 of 130319
Running 125334 of 130319
Running 125335 of 130319
Running 125336 of 130319
Running 125337 of 130319
Running 125338 of 130319
Running 125339 of 130319
Running 125340 of 130319
Running 125341 of 130319
Running 125342 of 130319
Running 125343 of 130319
Running 125344 of 130319
Running 125345 of 130319
Running 125346 of 130319
Running 125347 of 130319
Running 125348 of 130319
Running 125349 of 130319
Running 125350 of 130319
Running 125351 of 130319
Running 125352 of 130319
Running 125353 of 130319
Running 125354 of 130319
Running 125355 of 130319
Running 125356 of 130319

In [ ]:
# Generate questions based on a given passage
model.to(device)
sample_passage = 'This is a sample passage.'
sample_answer = 'Sample answer.'
input_text = f'generate questions: "{sample_passage}" answer: "{sample_answer}"'
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids=input_ids, max_length=64, num_beams=4, no_repeat_ngram_size=2, early_stopping=True)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

RuntimeError: ignored

In [ ]:
# Print the generated questions
print(output_text)

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/spiece.model',
 'fine_tuned_model/added_tokens.json')

In [ ]:
# Define the training function
def trainForQuestionGeneration(input_str, target_str):
    # Tokenize the input and output strings
    # Tokenize the input and target using the tokenizer
    input_ids = tokenizer(input_str, padding=True, truncation=True, return_tensors='pt').input_ids
    target_ids = tokenizer(target_str, padding=True, truncation=True, return_tensors='pt').input_ids

    input_len = len(input_ids[0])
    target_len = len(target_ids[0])

    # Set the model to training mode and zero the gradients
    model.train()
    optimizer.zero_grad()

    # Generate the output using the T5 model
    output_ids = model.generate(input_ids, max_length=input_len+target_len, do_sample=True)

    # Calculate the loss and perform backpropagation
    loss = torch.nn.functional.cross_entropy(output_ids[:, input_len:], target_ids[0])
    loss.backward()
    optimizer.step()

    # Return the predicted output
    return tokenizer.decode(output_ids[0, input_len:], skip_special_tokens=True)

In [ ]:
# Define the input loop
while True:
    # Get user input
    input_str = input('Context: ')
    question_str = input('Question: ')
    answer_str = input('Answer: ')

    # Combine the context and question into a single input string
    input_str = f"{input_str} <extra_id_0> {question_str} </s>"

    # Fine-tune the model and get the predicted answer
    predicted_answer = trainForQuestionGeneration(input_str, answer_str)

    # Print the predicted answer
    print(f"Predicted Answer: {predicted_answer}")

In [ ]:
# Load the required libraries
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
import torch

# Load the pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Load the SQuAD v2 dataset
dataset = load_dataset('squad_v2')

# Extract the passages and answers from the dataset
passages = dataset['train']['context']
answers = dataset['train']['answers']['text']

# Prepare the input and output for the T5 model
inputs = []
outputs = []
for i in range(len(passages)):
    input_text = f'generate questions: "{passages[i]}" answer: "{answers[i]}"'
    output_text = f'{dataset["train"][i]["question"]}'
    inputs.append(input_text)
    outputs.append(output_text)

# Tokenize the input and output using the T5 tokenizer
input_ids = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')
output_ids = tokenizer(outputs, padding=True, truncation=True, return_tensors='pt')

# Fine-tune the T5 model on the SQuAD v2 dataset
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
for epoch in range(3):
    for i in range(len(inputs)):
        input_ids_batch = input_ids[i].unsqueeze(0)
        output_ids_batch = output_ids[i].unsqueeze(0)
        loss = model(input_ids_batch, labels=output_ids_batch).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Generate questions based on a given passage
sample_passage = 'This is a sample passage.'
sample_answer = 'Sample answer.'
input_text = f'generate questions: "{sample_passage}" answer: "{sample_answer}"'
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids=input_ids, max_length=64, num_beams=4, no_repeat_ngram_size=2, early_stopping=True)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated questions
print(output_text)
